<a href="https://colab.research.google.com/github/FRED984/Line_follower_robot_CNN/blob/main/CNN_line_follower_robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#packages
from keras.models import Sequential
from keras.layers.convolutional  import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
import numpy as np
import argparse
import cv2
import os
import matplotlib


class LeNet :
    @staticmethod
    def build(width,height, depth, classes):
        #initialization du model
        model = Sequential()
        inputShape = (height, width , depth)


        #1ere couche conv=>RELU=> Pool

        model.add(Conv2D(20, (5,5), padding="same", input_shape=inputShape))#nb filtre:20,taille filtre:5,5 ,padding:same,taille entree:parametre fonction
        model.add(Activation("relu"))#fonction d'activation Relu
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))#pool:max ,taile filtre pooling:2,2;pas:2,2

        #2e couche
        model.add(Conv2D(50, (5,5), padding="same", input_shape=inputShape))#meme sauf nbre de filtre :50
        model.add(Activation("relu"))#meme
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))#meme que premier

        #couche fully connected
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        #softmax classifier
        model.add(Dense(classes))#fonction de classification Softmax
        model.add(Activation("softmax"))

        # return
        return model

Loading the image to the model

In [ ]:
from imutils import paths
import random
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.utils import img_to_array





In [ ]:
#path of the image
dataset ='/content/drive/MyDrive/Colab Notebooks/train'

#initialisation des donées
print("[Info] loading images...")
data=[]
labels= []

#shuffle des images
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)
#dataset split
trainX=[]
trainY=[]
testX=[]
testY=[]

for imagePath in imagePaths:
    image =cv2.imread(imagePath)
    image =cv2.resize(image,(28,28))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    print(label)
    if label == 'forwad':
        label =0
    elif label == 'right' :
        label = 1
    else:
        label =2
    labels.append(label)


data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
#partage des datas sous forme de tableau numpy  entre trainX,trainY,testX,testY
trainX, testX, trainY, testY= train_test_split(data, labels, test_size=0.3, random_state=42)
trainY = to_categorical(trainY, num_classes=3)
testY = to_categorical(testY, num_classes=3)







[Info] loading images...
left
right
forward
forward
left
forward
forward
forward
right
forward
right
forward
right
left
right
forward
right
forward
forward
left
right
backward
left
forward
forward
right
right
right
forward
forward
forward
forward
left
forward
forward
forward
right
backward
forward
forward
forward
forward
forward
forward
forward
forward
left
right
forward
left
forward
right
forward
forward
left
left
backward
left
left
right
forward
right
right
left
forward
right
right
forward
forward
right
right
backward
left
forward
right
forward
forward
forward
forward
forward
backward
forward
right


training the model

In [ ]:
from keras import backend  as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [ ]:
from keras.callbacks import History
EPOCHS = 15
INIT_LR = 1e-3
BS = 32


#initialisation du model
print("[INFO] compiling model...")
modele = LeNet.build(width=28, height=28, depth=3, classes=3)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)#algorithme d'optimisaton Adam
modele.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

#train the  network
print("[INFO] training network ...")
History= modele.fit(trainX, trainY,batch_size=BS ,epochs=EPOCHS, verbose=1)

#save the model to disk
print("[INFO] serializing network...")


[INFO] compiling model...
[INFO] training network ...
Epoch 1/15


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


2/2 [==============================] - 1s 83ms/step - loss: 0.5553 - accuracy: 0.4828
Epoch 2/15
2/2 [==============================] - 0s 80ms/step - loss: 0.3554 - accuracy: 0.7241
Epoch 3/15
2/2 [==============================] - 0s 75ms/step - loss: 0.2620 - accuracy: 0.9310
Epoch 4/15
2/2 [==============================] - 0s 77ms/step - loss: 0.1614 - accuracy: 0.9310
Epoch 5/15
2/2 [==============================] - 0s 69ms/step - loss: 0.0896 - accuracy: 0.9828
Epoch 6/15
2/2 [==============================] - 0s 81ms/step - loss: 0.0394 - accuracy: 1.0000
Epoch 7/15
2/2 [==============================] - 0s 74ms/step - loss: 0.0260 - accuracy: 0.9828
Epoch 8/15
2/2 [==============================] - 0s 84ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 9/15
2/2 [==============================] - 0s 94ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 10/15
2/2 [==============================] - 0s 67ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 11/15
2/2 [=======================

In [ ]:
y_pred = modele.predict(testX)

1/1 [==============================] - 0s 127ms/step


In [ ]:
result = modele.evaluate(testX, testY, verbose=1)

accuracy = result[1]

print(f"La précision de notre modèle est de {round(accuracy*100,2)} %")

1/1 [==============================] - 0s 187ms/step - loss: 2.2962e-04 - accuracy: 1.0000
La précision de notre modèle est de 100.0 %
